In [1]:
%cd C:\Users\New\PycharmProjects\underdog-devs-ds-a\

C:\Users\New\PycharmProjects\underdog-devs-ds-a


In [2]:
#!dir

In [3]:
import pandas as pd
import plotly.express as px
import altair as alt
from pandas import DataFrame

In [4]:
from app.data import MongoDB

In [5]:
db = MongoDB()

In [6]:
db

In [7]:
mentee_referrals = pd.DataFrame(
    db.projection('Mentees', {'is_active': True, 'validate_status': 'approved'}, {'referred_by': True}))

In [8]:
mentees = pd.DataFrame(db.read('Mentees'))

In [9]:
mentees['role'] = 'mentee'

In [10]:
mentees_activity = mentees.filter(['role', 'is_active'])

In [11]:
mentees_activity

,role,is_active
0,mentee,False
1,mentee,True
2,mentee,True
3,mentee,True
4,mentee,True
...,...,...
95,mentee,False
96,mentee,True
97,mentee,False
98,mentee,True


In [12]:
mentees_activity

,role,is_active
0,mentee,False
1,mentee,True
2,mentee,True
3,mentee,True
4,mentee,True
...,...,...
95,mentee,False
96,mentee,True
97,mentee,False
98,mentee,True


In [13]:
mentors = pd.DataFrame(db.read('Mentors'))

In [14]:
mentors = mentors.query("first_name != 'string'")

In [15]:
mentors['role'] = 'mentor'

In [16]:
mentors_activity = mentors.filter(['role', 'is_active'])

In [17]:
mentors_activity

,role,is_active
0,mentor,True
1,mentor,True
2,mentor,True
3,mentor,True
4,mentor,True
5,mentor,True
6,mentor,True
7,mentor,True
8,mentor,True
9,mentor,True


In [18]:
activity = pd.concat([mentees_activity, mentors_activity])

In [19]:
activity

,role,is_active
0,mentee,False
1,mentee,True
2,mentee,True
3,mentee,True
4,mentee,True
...,...,...
15,mentor,True
16,mentor,True
17,mentor,True
18,mentor,True


In [20]:
meetings_df = pd.DataFrame(db.read("Meetings"))

In [62]:
meetings_missed = meetings_df.filter(['meeting_missed_by_mentee'])

In [63]:
missed = alt.Chart(meetings_missed).mark_bar().encode(x = 'meeting_missed_by_mentee', y = 'count(meeting_missed_by_mentee)')

In [64]:
missed

alt.Chart(...)

In [65]:
activity_vis = alt.Chart(activity).mark_bar().encode(x = 'is_active', y = 'count(is_active)', color = 'role')

In [66]:
activity_vis

alt.Chart(...)